# CLUSTERS POLYGONS MONITORING: METHODS AND ENDPOINTS 

In [1]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


#### Custom Polygon Class

In [26]:
# CLUSTERS POLYGONS CLASS

# ---
# import modules

from warnings import filterwarnings as fws; fws('ignore')
import requests, json, pandas as pd, numpy as np
from matplotlib.path import Path as mpl_path
from copy import deepcopy

# ---
# custom modules
from modules.geojson_conversion import polygon_geojson

# ---
#### Polygon defaults

geometry_cols = ['lng_min', 'lng_max', 'lat_min', 'lat_max']

# ---
#### Polygon Methods

def filter_df(df, filters={}):
    df = deepcopy(df)
    for key, value in filters.items():
        if value is not list: value = [str(value)]
        df = df[df[key].astype(str).isin(value)]
    return df

# ---
#### Custom Polygon Class

class Polygon:

    def __init__(self, polygons, poly_id, base_id=None, urls=None, logic=None, drop_from_status=[], geometry_cols=geometry_cols):
        if type(polygons) is pd.DataFrame:
            self.polygons_df = polygons
            polygons = polygon_geojson(polygons, coords=geometry_cols)
        else:
            self.polygons_df = None
        self.polygons = polygons
        self.multipolygon = list(map(lambda feat: feat['geometry']['coordinates'], polygons['features']))
        self.poly_id = poly_id
        self.base_id = base_id
        if base_id is None:
            self.base_id = self.poly_id
        self.poly_ids = [feature['properties'][poly_id] for feature in polygons['features']]
        self.urls = urls
        self.logic = logic
        if logic is not None:
            self.labels = pd.Series([stage['fields'] for stage in logic]).sum()
        self.drop_from_status = drop_from_status
        self.geometry_cols = geometry_cols
        self.current_status = None

    def update_status(self):
        self.current_status = self.status()

    def logical_status(self, status=None):
        if status is None:
            if self.current_status is not None: status = self.current_status
            else: status = self.status()
        df = pd.DataFrame([[0, self.logic[0]['name']]]*len(status), columns=['status_code', 'status_name'], index=status.index)
        for code, state in enumerate(self.logic):
            is_state = status[state['fields']].sum(1) > 0
            df[is_state] = [code, state['name']]
        return df

    def status_from_url(self, url, params):
        url_split = url.split('/')
        root, subpath = '/'.join(url_split[:2]), '/'.join(url_split[3:])
        df = pd.DataFrame(requests.get(url).json())
        if 'prefix' not in params.keys():
            params['prefix'] = '_'.join(subpath.split('/'))
        return self.has_points(df, params['id'], params['coords'].split(','), params['prefix'])
    
    # Get and append current monitoring data to json
    def status(self, drop_geometry=False):
        dfs = []
        for url in self.urls:
            df = pd.DataFrame(requests.get(url['url']).json())
            if 'coords' not in url.keys():
                if 'prefix' in url.keys():
                    df.add_prefix(url['prefix']+'_')
                dfs.append(df.set_index(url['id']))
            else:
                if 'filters' not in url.keys():
                    url['filters'] = [{'params': {}, 'prefix': url['prefix']}]
                for filters in url['filters']:
                    df_subset = filter_df(df, filters['params']) # Test if original is not changing ###
                    df_poly = self.has_points(df_subset, url['id'], url['coords'], filters['prefix'])
                    dfs.append(df_poly.set_index(self.base_id))
        status = self.polygons_df.set_index(self.poly_id).join(dfs).reset_index().rename(columns={self.poly_id: self.base_id})
        outcols = self.drop_from_status + (self.geometry_cols if drop_geometry else [])
        outcols = [col for col in outcols if col in status]
        self.current_status = status
        if self.logic is not None: status = self.logical_status().join(status)
        if len(outcols): status.drop(outcols, axis=1, inplace=True)
        self.current_status = status
        return status

    def general_status(self, status=None, drop_geometry=False):
        if status is None:
            if self.current_status is not None: status = self.current_status
            else: status = self.status()
        if self.logic is None: raise 'Must provide class "logic" parameter to get general status'
        poly_status = status
        code = poly_status['status_code'].max()
        counts = poly_status['status_name'].value_counts().to_dict()
        for stage in self.logic:
            if stage['name'] not in counts.keys():
                counts[stage['name']] = 0
        fields = []
        for url in self.urls:
            if 'filters' in url:
                if len(url['filters']):
                    for field in url['filters']:
                        fields.append(field['prefix'])
            elif 'prefix' in url.keys():
                fields.append(url['prefix'])
        fields = pd.Series(fields)
        ctgr_cnts = poly_status[fields + '_count'].sum()
        ctgr_sts = (poly_status[fields + '_status'].sum() > 0).astype(int)
        ctgr_ids = poly_status[fields + '_ids'].sum()
        gen_status = pd.DataFrame([{
            'status_code': code,
            'status_name': self.logic[code]['name'],
            **counts, **ctgr_sts,
            **ctgr_cnts, **ctgr_ids,
            'coordinates': self.multipolygon
        }])
        if drop_geometry:
            del gen_status['coordinates']
        return gen_status
    
    # Returns polygon id list given a points list and a polygons geojson object    
    def points_belong(self, df, point_id, coord_cols):
        if not len(df): return pd.Series()
        # Get events points array
        points = np.array(list(map(tuple, df[coord_cols].values)))
        # Get clusters polygons events dict
        events_poly = {}
        for feature in self.polygons['features']:
            cluster_id = feature['properties'][self.poly_id]
            poly = feature['geometry']['coordinates'][0]
            mpl_poly =  mpl_path(poly)
            points_msk = mpl_poly.contains_points(points)
            poly_events_df = df[points_msk]
            poly_events_ids = list(poly_events_df[point_id])
            for event_id in poly_events_ids:
                events_poly[event_id] = cluster_id
        # Update events dataframe with events polygons ids
        belong_msk = pd.Series(- np.ones(len(df)), index=df.index)
        for event_id, cluster_id in events_poly.items():
            belong_msk[df[point_id]==event_id] = cluster_id
        # Return extended open events
        return belong_msk.fillna('')

    # Appends 'polygon has points' columns to polygons dataset
    def has_points(self, df, point_id, coord_cols, prefix=''):
        points_poly_ids = self.points_belong(df, point_id, coord_cols)
        has_df = []
        for _id in self.poly_ids:
            id_msk = points_poly_ids == _id
            n_points = sum(id_msk)
            has_points = int(n_points > 0)
            points_ids = list(df[id_msk][point_id])
            has_df.append([_id, n_points, has_points, points_ids])
        has_df_cols = [self.base_id] + [prefix + '_' + col for col in ['count', 'status', 'ids']]
        return pd.DataFrame(has_df, columns=has_df_cols)

---

# CLASS INSTANCE USAGE

## Get external data for testing

#### Import custom modules

In [3]:
from modules.mongo_requests import last_prediction_record
from modules.comando import comando_open_events #, polygons_geojson
from modules.waze import get_waze_partner_alerts
from modules.geojson_conversion import polygon_geojson

#### Comando pops dataset

In [4]:
# pop id map
comando_root = 'https://api.dados.rio/v2/adm_cor_comando'
pops_url = comando_root + '/pops'
pops_df = pd.DataFrame(requests.get(pops_url).json()['objeto'])
pops_map = pops_df.set_index('id')['titulo'].to_dict()

#### Sample points dataset

In [5]:
comando = comando_open_events()
waze = get_waze_partner_alerts()

Comando open events: Request attempt (1) 


---

## Polygons monitoring

### Polygons instance parameters settings

In [6]:
# ---
# urls monitoring parameters

api_root = 'http://127.0.0.1:5000'

urls = [{
    'url': '/predict',
    'id': 'cluster_id',
}, {
    'url': '/waze/alerts',
    'id': 'uuid',
    'coords': ['longitude', 'latitude'],
    'prefix': 'waze',
    'filters': [{
        'params': {'type': 'HAZARD_WEATHER_FLOOD'},
        'prefix': 'waze_flood'
    }]
}, {
    'url': '/comando/events',
    'id': 'id',
    'coords': ['longitude', 'latitude'],
    'prefix': 'comando',
    'filters': [{
        'params': {'pop_id': 33},
        'prefix': 'lâmina'
    }, {
        'params': {'pop_id': 5},
        'prefix': 'bolsão'
    }, {
        'params': {'pop_id': 31},
        'prefix': 'alagamento'
    }, {
        'params': {'pop_id': 6},
        'prefix': 'alagamento_enchente'
    }, {
        'params': {'pop_id': 32},
        'prefix': 'enchente'
    }, {
        'params': {'pop_id': 16},
        'prefix': 'vazamento'
    }, {
        'params': {'pop_id': 30},
        'prefix': 'sirene'
    }]
}]

for url in urls: url['url'] = api_root + url['url']

# ---
# logic states parameters

logic = [{
    'name': 'NORMALIDADE',
    'fields': []
}, {
    'name': 'ATENÇÃO',
    'fields': ['label', 'waze_flood_status']
}, {
    'name': 'ALERTA',
    'fields': ['lâmina_status', 'vazamento_status', 'sirene_status']    
}, {
    'name': 'PERIGO',
    'fields': ['bolsão_status', 'alagamento_status', 'alagamento_enchente_status', 'enchente_status']
}]

# ---
# columns parameters

geometry_cols = ['lng_min', 'lng_max', 'lat_min', 'lat_max']

clusters_drop = [
    'radius', 'horizontal_perimeter', 'density_circle', 'density_box',
    'vertical_perimeter', 'lng_center', 'title', 'lat_center',
    'lat_centroid', 'lng_centroid', 'area_circle'
]

prob_drop = ['_id', 'cluster', 'range']
comando_drop = []
waze_drop = []

drop_from_status = clusters_drop + prob_drop + comando_drop + waze_drop

# ---
# Load polygons dataset

polygons_df = pd.read_csv('static/clusters/clusters_micro.csv')

# ---
# Base dataframes for 'Polygon' class instances
city_polygon_df = polygons_df[polygons_df['sublabel'] == -1]
polygons_df = polygons_df[polygons_df['sublabel'] != -1]

### Get polygon class instances

In [27]:
polygons = Polygon(
    polygons_df, poly_id='sublabel', base_id='cluster_id', urls=urls, logic=logic,
    drop_from_status=drop_from_status, geometry_cols=geometry_cols
)

city_polygon = Polygon(
    city_polygon_df, poly_id='sublabel', base_id='cluster_id', urls=urls, logic=logic,
    drop_from_status=drop_from_status, geometry_cols=geometry_cols
)

### Get comando points polygons ids

In [8]:
point_id = 'id'
coord_cols = ['latitude', 'longitude']
prefix = 'comando'

comando['cluster_id'] = polygons.points_belong(comando, point_id, coord_cols)

comando['cluster_id'].head()

0   -1.0
1   -1.0
2   -1.0
3   -1.0
4   -1.0
Name: cluster_id, dtype: float64

### Get polygons points info

In [9]:
comando_status = polygons.has_points(comando, point_id, coord_cols, prefix=prefix)

comando_status.head()

,cluster_id,comando_count,comando_status,comando_ids
0,0,0,0,[]
1,1,0,0,[]
2,2,0,0,[]
3,3,0,0,[]
4,4,0,0,[]


### Requests dinamic polygons monitoring

In [10]:
status = polygons.status(drop_geometry=True)

status.head()

,cluster_id,main_neighborhood,main_route,main_street_number_range,label_count,area_box,timestamp,date,time,probability,...,enchente_status,enchente_ids,vazamento_count,vazamento_status,vazamento_ids,sirene_count,sirene_status,sirene_ids,status_code,status_name
0,0,Barra da Tijuca,Avenida Armando Lombardi,3098 - 67,114,0.047620,2023-01-01 15:20:00.044000,2023-01-01,12:20:00,0.036589,...,0,[],0,0,[],0,0,[],0,NORMALIDADE
1,1,Catete,Rua do Catete,228 - 139,99,0.057771,2023-01-01 15:20:00.044000,2023-01-01,12:20:00,0.002233,...,0,[],0,0,[],0,0,[],0,NORMALIDADE
2,2,Copacabana,Rua Tonelero,236 - 9,43,0.040122,2023-01-01 15:20:00.044000,2023-01-01,12:20:00,0.006931,...,0,[],0,0,[],0,0,[],0,NORMALIDADE
3,3,Ipanema,Avenida Epitácio Pessoa,1910 - 1602,36,0.039586,2023-01-01 15:20:00.044000,2023-01-01,12:20:00,0.001090,...,0,[],0,0,[],0,0,[],0,NORMALIDADE
4,4,Barra da Tijuca,Avenida Ministro Ivan Lins,1770 - 11,36,0.052712,2023-01-01 15:20:00.044000,2023-01-01,12:20:00,0.050550,...,0,[],0,0,[],0,0,[],0,NORMALIDADE


### Polygons logical status

#### Built in status columns

In [11]:
status[['status_code', 'status_name']].head()

,status_code,status_name
0,0,NORMALIDADE
1,0,NORMALIDADE
2,0,NORMALIDADE
3,0,NORMALIDADE
4,0,NORMALIDADE


#### Instance method

In [12]:
logic_status = polygons.logical_status()

logic_status.head()

,status_code,status_name
0,0,NORMALIDADE
1,0,NORMALIDADE
2,0,NORMALIDADE
3,0,NORMALIDADE
4,0,NORMALIDADE


### Status from url

In [13]:
url_status = polygons.status_from_url(url=api_root+'/comando/events', params={'id':'id', 'coords': 'longitude,latitude'})

url_status.head()

,cluster_id,comando_events_count,comando_events_status,comando_events_ids
0,0,0,0,[]
1,1,0,0,[]
2,2,0,0,[]
3,3,0,0,[]
4,4,0,0,[]


### Polygons 'general status'

#### Polygons general status

In [30]:
gen_status = polygons.general_status(drop_geometry=False)

gen_status

,status_code,status_name,NORMALIDADE,ATENÇÃO,ALERTA,PERIGO,waze_flood_status,lâmina_status,bolsão_status,alagamento_status,...,sirene_count,waze_flood_ids,lâmina_ids,bolsão_ids,alagamento_ids,alagamento_enchente_ids,enchente_ids,vazamento_ids,sirene_ids,coordinates
0,0,NORMALIDADE,79,0,0,0,0,0,0,0,...,0,[],[],[],[],[],[],[],[],"[[[[-43.3128293, -23.0071335], [-43.3128293, -..."


#### Polygons general status to geojson

In [31]:
gen_status_geojson = polygon_geojson(gen_status, coords='coordinates', geometry_type='MultiPolygon')

# gen_status_geojson

#### City polygon status

In [22]:
city = city_polygon.status(drop_geometry=True)

city

,cluster_id,main_neighborhood,main_route,main_street_number_range,label_count,area_box,timestamp,date,time,probability,...,enchente_status,enchente_ids,vazamento_count,vazamento_status,vazamento_ids,sirene_count,sirene_status,sirene_ids,status_code,status_name
0,-1,Barra da Tijuca,Avenida Brasil,35025 - 14,1624.0,1595.276593,NaN,NaN,NaN,NaN,...,0.0,[],3.0,1.0,"[87152, 87313, 86877]",0.0,0.0,[],2,ALERTA


#### City polygon status to geojson

In [25]:
city_geojson = polygon_geojson(city_polygon.status())

city_geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-43.692051, -23.031018600000003],
      [-43.692051, -22.7636397],
      [-43.1669444, -22.7636397],
      [-43.1669444, -23.031018600000003],
      [-43.692051, -23.031018600000003]]]},
   'properties': {'cluster_id': -1,
    'main_neighborhood': 'Barra da Tijuca',
    'main_route': 'Avenida Brasil',
    'main_street_number_range': '35025 - 14',
    'label_count': 1624.0,
    'area_box': 1595.2765933946553,
    'timestamp': nan,
    'date': nan,
    'time': nan,
    'probability': nan,
    'confidence': nan,
    'label': nan,
    'waze_flood_count': 0.0,
    'waze_flood_status': 0.0,
    'waze_flood_ids': [],
    'lâmina_count': 0.0,
    'lâmina_status': 0.0,
    'lâmina_ids': [],
    'bolsão_count': 0.0,
    'bolsão_status': 0.0,
    'bolsão_ids': [],
    'alagamento_count': 0.0,
    'alagamento_status': 0.0,
    'alagamento_ids': [],
    'alagamento_enchente_count